# 銀イルカの3人のセリフをスクレイピングして，同じところと違うところを

とりま wikiとかから智絵里のセリフをスクレイピングする． 

テキスト解析 -> 出現頻度，


セリフをword2vecで学習して，分散表現を計算できるようにする

分散表現を次元削減して2次元座標上にプロットすることで，単語ごとの類似度を可視化する

銀イルカ3人のセリフをスクレイピング  

doc2vecで学習して，

In [1]:
import re
import time
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
pd.options.display.max_rows = 500

## デレステセリフのスクレイピング

In [3]:
def get_quotes_sute(q):
    """
    デレステで公開されている各アイドルのセリフデータを取得する
    [arg]
    q: int
    [ret]
    df: DataFrame
    """
    html = f"https://starlight.kirara.ca/char/{q}"
    r = requests.get(html)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text, "lxml")
    texts = soup.find_all("span")
    corpus = []
    for text in texts:
        corpus.append(text.text)
    df = pd.DataFrame(corpus)
    #df[0].value_counts().head(10) # 値の出現頻度をカウント
    df = df[~df.duplicated()] # 重複行をdataframeから除外
    df = df[~df[0].str.match("^[\s|a-zA-Z|<|\+|\d$]")] # 正規表現にマッチするものを除外
    df = df[df[0] != ""]
    return df.reset_index(drop=True)

## モバマスセリフのスクレイピング

In [4]:
def get_quotes_moba(query="緒方智絵里", debug=False):
    """
    モバマスで公開されているセリフ（含 アイドル名、カード名 など）
    をスクレイピングして pandas dataframe にする
    
    [args]
    query: str 省略すると 緒方智絵里 のセリフをスクレイピングする
    debug: Bool Trueにするとスクレイピング数を上限2にする。それだけ もっと良い使い方あるはず...
    [ret]
    df: DataFrame
    """
    
    html = f"https://icws.indigo-bell.com/search?q={query}&st=n"
    r = requests.get(html)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text, "lxml")
    a = soup.find("a", string="Last　»").get("href")
    if debug == True:
        n = 2
    else:
        n = int(re.search(r"page=(\d+)&", a).group(1)) # クロールするページ数
    
    df = pd.DataFrame()
    for i in range(n): # セリフが登録されてるページ分ループする
        print(f"crawling page: {i + 1}/{n}")
        html = f"https://icws.indigo-bell.com/search?page={i + 1}&q={query}&st=n"
        r = requests.get(html)
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "lxml")
        table = soup.find_all("tbody",class_="result")[0]
        rows = table.find_all("tr")
        tmp = []
        tm = 5 + random.randint(0, 5)

        for row in rows:
            cell_txt = []
            for cell in row.find_all(["td", "th"]):
                cell_txt.append(cell.text)
            tmp.append(cell_txt)
        df = df.append(tmp)
        #print(f"sleep: {tm}")
        if i != (n - 1):
            time.sleep(tm)
        else:
            continue
    return df.reset_index(drop=True) # indexを振り直してもとのindexも削除

In [5]:
# deresute id
# chieri : 112
# minami : 185
# yui    : 246
# sae    : 132
# hotaru : 161
# yoshino: 262

In [6]:
aa = get_quotes_sute(112)

In [7]:
bb = get_quotes_moba(query="緒方智絵里", debug=True)

crawling page: 1/2
crawling page: 2/2


In [8]:
aa.head(5)

,0
0,アイドルマスター シンデレラガールズ スターライトステージ!
1,お、おはようございます。アーリーモーニングティー、いかがですか…？
2,お昼だけど…もうちょっとだけ…午後のレッスンの予習をしなくっちゃ…
3,ふぁぁ…こんな時間までお疲れさまです。ホットミルク、飲みますか…？
4,プロデューサーさん…もしお暇なら、私…ステージに、立ちたい…です


In [9]:
bb.tail(5)

,0,1,2,3
95,緒方智絵里,[ﾊﾞﾚﾝﾀｲﾝ]緒方智絵里+,お仕事,〇〇さん、わたし、一緒だとラッキーかもしれないです…!
96,緒方智絵里,[ﾊﾞﾚﾝﾀｲﾝ]緒方智絵里+,お仕事,えへっ…うまく渡せました!
97,緒方智絵里,[ﾊﾞﾚﾝﾀｲﾝ]緒方智絵里+,お仕事,わ、わたしにファンっているのかな
98,緒方智絵里,[ﾊﾞﾚﾝﾀｲﾝ]緒方智絵里+,お仕事（親愛度UP）,わたし…今なら言えそうな気がします…で、でも…まだ秘密っ
99,緒方智絵里,[ﾊﾞﾚﾝﾀｲﾝ]緒方智絵里+,LIVEバトル,負けませんからっ


## モバ，ステの結合と正規化

In [10]:
bb[3] = bb[3].str.replace("○○", "PRODUCER") # OO (プロデューサー名)部分を 置換して統一
# aa + bb のコーパスを作る
docs = pd.Series()
docs = pd.concat([aa[0], bb[3]])
docs = docs.str.replace("プロデューサー", "PRODUCER")
docs = docs.str.replace("[Producer]", "PRODUCER", regex=False)
docs = docs.reset_index(drop=True)

## 形態素解析

mecabを使う。インストールしていなければめかぶをインストール

neeologdまでインストールする手順はの参考は
- [ubuntu 18.10 に mecab をインストール](https://qiita.com/ekzemplaro/items/c98c7f6698f130b55d53)
- [WSLでMeCabとJupyter Notebookを動かす](https://qiita.com/hironobukawaguchi3/items/61a8664dd26c1da66880)
- [MecabをPythonで使うまで](https://qiita.com/Sak1361/items/47e9ec464ccc770cd65c)

---

$ mecab -d /usr/lib/mecab/dic/mecab-ipadic-neologd

In [11]:
def tokenize(text, dic="neologd"):
    """
    テキストを形態素解析して分かち書きする
    [arg]
    text: str
    [ret]
    term: str
    """
    import MeCab
    if dic == "neologd":
        wakati = MeCab.Tagger('-Owakati -d /usr/lib/mecab/dic/mecab-ipadic-neologd')
    elif dic == "ipa":
        wakati = MeCab.Tagger('-Owakati')
    term = wakati.parse(text).strip()
    return term

In [12]:
print(f'ipa-dic    ： {tokenize(docs[95])}')
print(f'neologd-dic： {tokenize(docs[95])}')

ipa-dic    ： PRODUCER さん が 見 て て くれる … よし 、 がんばろう っ …
neologd-dic： PRODUCER さん が 見 て て くれる … よし 、 がんばろう っ …


In [13]:
from gensim import corpora, models

In [25]:
docs_list = [tokenize(row).split() for row in docs]
dic = corpora.Dictionary(docs_list)
dic.save_as_text('./data/text.dict')  # 保存
#gensim.corpora.Dictionary.load_from_text('./data/text.dict')  # 次回からファイルロード可

In [35]:
print(len(dic)) # dictionaryの形態素数
#print(dic.token2id) # tokenとidの対応

1719


In [79]:
# 単語出現頻度
# 余談: bashだと $ sort -t$'\t' -k3 -nr text.dict で単語とその出現頻度回数カウントできる
sorted(dic.items(), key=lambda x:x[0])

[(0, '!'),
 (1, 'アイドルマスター'),
 (2, 'シンデレラガールズ'),
 (3, 'スターライトステージ'),
 (4, '…'),
 (5, '、'),
 (6, '。'),
 (7, 'いかが'),
 (8, 'お'),
 (9, 'おはよう'),
 (10, 'か'),
 (11, 'ござい'),
 (12, 'です'),
 (13, 'ます'),
 (14, 'アーリーモーニング'),
 (15, 'ティー'),
 (16, '？'),
 (17, 'お昼'),
 (18, 'けど'),
 (19, 'し'),
 (20, 'だ'),
 (21, 'だけ'),
 (22, 'ちゃ'),
 (23, 'ちょっと'),
 (24, 'なくっ'),
 (25, 'の'),
 (26, 'もう'),
 (27, 'を'),
 (28, 'レッスン'),
 (29, '予習'),
 (30, '午後'),
 (31, 'ぁ'),
 (32, 'お疲れさま'),
 (33, 'こんな'),
 (34, 'ふぁ'),
 (35, 'まで'),
 (36, 'ホットミルク'),
 (37, '時間'),
 (38, '飲み'),
 (39, 'PRODUCER'),
 (40, 'さん'),
 (41, 'たい'),
 (42, 'なら'),
 (43, 'に'),
 (44, 'もし'),
 (45, 'ステージ'),
 (46, '暇'),
 (47, '私'),
 (48, '立ち'),
 (49, 'あの'),
 (50, 'い'),
 (51, 'え'),
 (52, 'が'),
 (53, 'た'),
 (54, 'ちひろ'),
 (55, 'で'),
 (56, 'と'),
 (57, 'まし'),
 (58, 'よ'),
 (59, '呼ん'),
 (60, 'う'),
 (61, 'そう'),
 (62, 'でしょ'),
 (63, 'ら'),
 (64, '宛て'),
 (65, '申請'),
 (66, '誰か'),
 (67, 'いる'),
 (68, 'さ'),
 (69, 'て'),
 (70, 'みたい'),
 (71, 'れ'),
 (72, 'わぁ'),
 (73, 'イベント'),
 (74, '楽し'),
 (7

In [ ]:
from collections import defaultdict
freq = defaultdict(int)

for text in docs_list:
    for token in text:
        freq[token] += 1

In [28]:
corpus = [dic.doc2bow(doc) for doc in docs_list]
corpora.MmCorpus.serialize('./data/text.mm', corpus)  # 保存

In [29]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

# 折角計算したのでpickleに保存
import pickle
with open('./data/corpus_tfidf.dump', mode='wb') as f:
    pickle.dump(corpus_tfidf, f)

# 次回からはロードできます
# with open('./data/corpus_tfidf.dump', mode='rb') as f:
#     corpus_tfidf = pickle.load(f)

In [30]:
# 文書量にもよりますが、数時間程度かかったりもします。
# '18/12/03追記: LdaMulticore で worker の数を増やせばかなり早くなるかもです
lda = models.LdaModel(corpus=corpus_tfidf, id2word=dic,
                             num_topics=50, minimum_probability=0.001,
                             passes=20, update_every=0, chunksize=10000)
lda.save('./data/lda.model')  # 保存
#lda = gensim.models.LdaModel.load('./data/lda.model')  # 次回からロード可

In [31]:
for i in range(10):
    print('tpc_{0}: {1}'.format(i, lda.print_topic(i)))

tpc_0: 0.029*"て" + 0.027*"の" + 0.026*"私" + 0.025*"た" + 0.023*"PRODUCER" + 0.022*"さん" + 0.021*"を" + 0.021*"ん" + 0.021*"。" + 0.020*"か"
tpc_1: 0.019*"いっ" + 0.018*"どこか" + 0.017*"天使" + 0.015*"あり" + 0.014*"先" + 0.013*"どんなに" + 0.012*"泣き" + 0.011*"わっ" + 0.010*"期待" + 0.010*"いる"
tpc_2: 0.018*"かな" + 0.015*"ちゃ" + 0.014*"いけ" + 0.013*"なく" + 0.012*"新世界" + 0.012*"景色" + 0.011*"なれる" + 0.011*"で" + 0.010*"じゃ" + 0.010*"ない"
tpc_3: 0.019*"すみません" + 0.017*"パンツ" + 0.017*"ほら" + 0.011*"もこもこ" + 0.010*"居場所" + 0.010*"めくる" + 0.009*"ほか" + 0.009*"なのか" + 0.009*"言え" + 0.009*"秘密"
tpc_4: 0.058*"わたし" + 0.017*"がんばろう" + 0.017*"PRODUCER" + 0.015*"だ" + 0.015*"よし" + 0.014*"手" + 0.014*"て" + 0.014*"くれる" + 0.013*"っ" + 0.013*"夢"
tpc_5: 0.019*"すぐ" + 0.015*"～" + 0.015*"いっぱい" + 0.014*"あるかな" + 0.011*"受け止め" + 0.010*"ちゃい" + 0.010*"息" + 0.010*"いつか" + 0.009*"冷め" + 0.009*"ぬるい"
tpc_6: 0.033*"あの" + 0.021*"あ" + 0.020*"ぇい" + 0.016*"なきゃ" + 0.015*"はぁ" + 0.013*"思っ" + 0.010*"振っ" + 0.010*"１" + 0.010*"０" + 0.010*"介抱"
tpc_7: 0.032*"冬" + 0.027*"好き" + 0.